<a href="https://colab.research.google.com/github/Speedbird45Bravo/Keras_projects_21/blob/main/Keras_41221.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
from keras.layers import Dense, Dropout
from keras.models import Sequential
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

df = pd.read_csv("https://projects.fivethirtyeight.com/soccer-api/club/spi_matches.csv").dropna()
df = df[(df.league=='Barclays Premier League') | (df.league=='English League Championship') | (df.league=='Major League Soccer')]
df['spi_margin'] = np.abs(df['spi1'] - df['spi2'])
df['margin'] = np.abs(df['score1'] - df['score2'])
df['goal_count'] = df['score2'] + df['score1']

gcount = []

for x in df['goal_count']:
  if x > 0:
    gcount.append("GOALS")
  else:
    gcount.append("SCORELESS")

df['goal_count_bucket'] = gcount

In [34]:
def load_model():
  model = Sequential()
  model.add(Dense(16, activation='relu'))
  model.add(Dense(2, activation='softmax'))
  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
  return model

def to_cat(array):
  y = to_categorical(np.asarray(pd.factorize(array)[0]))
  return y

In [35]:
X = df[['score1','score2']].reset_index(drop=True)
y = df[['goal_count_bucket']].copy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=6)
y_train = to_cat(y_train['goal_count_bucket'])
y_test = to_cat(y_test['goal_count_bucket'])

In [36]:
model = load_model()
model.fit(X_train, y_train, epochs=40, batch_size=128, verbose=1, validation_split=0.2)

Epoch 1/40
26/26 [==============================] - 1s 9ms/step - loss: 0.9727 - accuracy: 0.2694 - val_loss: 0.8236 - val_accuracy: 0.3204
Epoch 2/40
26/26 [==============================] - 0s 3ms/step - loss: 0.7781 - accuracy: 0.3892 - val_loss: 0.6590 - val_accuracy: 0.6408
Epoch 3/40
26/26 [==============================] - 0s 3ms/step - loss: 0.6286 - accuracy: 0.6987 - val_loss: 0.5343 - val_accuracy: 0.9175
Epoch 4/40
26/26 [==============================] - 0s 4ms/step - loss: 0.5066 - accuracy: 0.9240 - val_loss: 0.4407 - val_accuracy: 0.9272
Epoch 5/40
26/26 [==============================] - 0s 3ms/step - loss: 0.4189 - accuracy: 0.9358 - val_loss: 0.3676 - val_accuracy: 0.9272
Epoch 6/40
26/26 [==============================] - 0s 3ms/step - loss: 0.3494 - accuracy: 0.9291 - val_loss: 0.3121 - val_accuracy: 0.9272
Epoch 7/40
26/26 [==============================] - 0s 3ms/step - loss: 0.2972 - accuracy: 0.9347 - val_loss: 0.2706 - val_accuracy: 0.9272
Epoch 8/40
26/26 [==

In [37]:
predictions = model.predict(X_test).round(0)

loss, accuracy = model.evaluate(y_test, predictions)
print("Test Accuracy: %.2f" % (accuracy * 100) + "%")

33/33 [==============================] - 0s 2ms/step - loss: 0.0614 - accuracy: 1.0000
Test Accuracy: 100.00%
